In [1]:
import numpy as np
import IPython.display

In [2]:
class Conv:
    def __init__(self, num_filters):
        self.num_filters = num_filters
        self.filters = np.random.randn(num_filters, 3, 3)/9
    def iterate_regions(self, image):
        h,w = image.shape
        for i in range(h-2):
            for j in range(w-2):
                im_region = image[i:(i+3), j:(j+3)]
                yield im_region, i, j
                
    def forward(self, input):
        self.last_input = input
        h,w = input.shape
        output = np.zeros((h-2, w-2, self.num_filters))
        for im_regions, i, j in self.iterate_regions(input):
            output[i, j] = np.sum(im_regions * self.filters, axis=(1,2))
        return output
    
    def backprop(self, d_l_d_out, learn_rate):
        d_l_d_filters = np.zeros(self.filters.shape)
        for im_region, i, j in self.iterate_regions(self.last_input):
            for f in range(self.num_filters):
                d_l_d_filters[f] += d_l_d_out[i,j,f] * im_region
        self.filters -= learn_rate * d_l_d_filters
        return None

In [3]:
class MaxPool:
    def iterate_regions(self, image):
        h, w, _ = image.shape  
        new_h = h // 2
        new_w = w // 2
        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i*2):(i*2+2), (j*2):(j*2+2)]
                yield im_region, i, j
    def forward(self, input):
        self.last_input = input
        h, w, num_filters = input.shape
        output = np.zeros((h//2, w//2, num_filters))
        for im_region, i, j in self.iterate_regions(input):
            output[i,j] = np.amax(im_region,axis=(0,1))
        return output
    
    def backprop(self, d_l_d_out):
        d_l_d_input = np.zeros(self.last_input.shape)
        for im_region, i, j in self.iterate_regions(self.last_input):
            h, w, f = im_region.shape
            amax = np.amax(im_region, axis=(0,1))
            for i2 in range(h):
                for j2 in range(w):
                    for f2 in range(f):
                        #if the pixel was the max value, copy the gradient to it
                        if(im_region[i2,j2,f2] == amax[f2]):
                            d_l_d_input[i*2+i2, j*2+j2 ,f2] = d_l_d_out[i, j, f2]
                            break;
        return d_l_d_input

In [4]:
class Softmax:
    def __init__(self, input_len, nodes):
        # We divide by input_len to reduce the variance of our initial values
        self.weights = np.random.randn(input_len, nodes)/input_len
        self.biases = np.zeros(nodes)
    def forward(self, input):
        self.last_input_shape = input.shape
        input = input.flatten()
        self.last_input = input
        input_len, nodes = self.weights.shape
        totals = np.dot(input, self.weights) + self.biases
        self.last_totals = totals
        exp = np.exp(totals)
        return(exp/np.sum(exp, axis=0)) 
    
    def backprop(self, d_l_d_out, learn_rate):
        for i, gradient in enumerate(d_l_d_out):
            if(gradient == 0):
                continue
            t_exp = np.exp(self.last_totals)
            S = np.sum(t_exp)
            d_out_d_t = -t_exp[i] * t_exp/ (S**2)
            d_out_d_t[i] = t_exp[i] * (S-t_exp[i]) /(S**2)
            
            d_t_d_w = self.last_input
            d_t_d_b = 1
            d_t_d_inputs = self.weights
            
            d_l_d_t = gradient * d_out_d_t
            
            d_l_d_w = d_t_d_w[np.newaxis].T @ d_l_d_t[np.newaxis]
            d_l_d_b = d_l_d_t * d_t_d_b  
            d_l_d_inputs = d_t_d_inputs @ d_l_d_t
            
            self.weights -= learn_rate * d_l_d_w
            self.biases -= learn_rate * d_l_d_b
            return d_l_d_inputs.reshape(self.last_input_shape)

In [9]:
train_file = 'E:/Curious Dev B/Kaggle/train.csv'
test_file = 'E:/Curious Dev B/Kaggle/test.csv'

def get_data_with_label(file):
    fh = open(file)
    ct = 0
    labels = []
    images = []
    for line in fh:
        ct += 1
        if ct == 1:
            continue
        pix_str = line.split(',')
        labels.append(float(pix_str[0]))
        pix_np = np.asarray(pix_str[1:], dtype = float)
        pix_np_split = np.array_split(pix_np, 28)
        pix_np_split_asarray = np.asarray(pix_np_split, dtype = 'int')
        images.append(pix_np_split_asarray)
    return images, labels

def get_data(file):
    fh = open(file)
    ct = 0
    images = []
    for line in fh:
        ct += 1
        if ct == 1:
            continue
        pix_str = line.split(',')
        pix_np = np.asarray(pix_str, dtype = float)
        pix_np_split = np.array_split(pix_np, 28)
        pix_np_split_asarray = np.asarray(pix_np_split, dtype = 'int')
        images.append(pix_np_split_asarray)
    return images

tr_img_list, tr_lb_list = get_data_with_label(train_file)
t_img_list = get_data(test_file)

training_images = np.asarray(tr_img_list)
training_labels = np.asarray(tr_lb_list)
testing_images = np.asarray(t_img_list)

print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)

training_labels = np.asarray(training_labels, dtype = 'int')

(42000, 28, 28)
(42000,)
(28000, 28, 28)


In [ ]:
conv = Conv(8)
pool = MaxPool()
softmax = Softmax(13 * 13 * 8, 10)

def forward(image, label):
    out = conv.forward((image/255) - 0.5)
    out = pool.forward(out)
    out = softmax.forward(out)
    loss = -np.log(out[label])
    acc = 1 if(np.argmax(out) == label) else 0
    
    return out, loss, acc

def train(im, label, lr=0.005):
    out,loss,acc = forward(im, label)
    
    gradient = np.zeros(10)
    gradient[label] = -1/out[label]
    
    gradient = softmax.backprop(gradient, lr)
    gradient = pool.backprop(gradient)
    gradient = conv.backprop(gradient, lr)
    
    return loss, acc
    
print('Training Starts....')

for epoch in range(3):
    print('----EPOCH %d ---'%(epoch+1))

    loss = 0
    num_correct = 0
    for i, (im, label) in enumerate(zip(training_images, training_labels)):
        if(i>0 and i %1000 == 999):
            print('[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %(i + 1, loss / 100, num_correct))
            loss = 0
            num_correct = 0
        l, acc = train(im, label)
        loss += l
        num_correct += acc

MNIST CNN initialized
----EPOCH 1 ---
[Step 100] Past 100 steps: Average Loss 2.243 | Accuracy: 16%
[Step 200] Past 100 steps: Average Loss 2.239 | Accuracy: 26%
[Step 300] Past 100 steps: Average Loss 2.002 | Accuracy: 37%
[Step 400] Past 100 steps: Average Loss 1.879 | Accuracy: 48%
[Step 500] Past 100 steps: Average Loss 1.511 | Accuracy: 66%
[Step 600] Past 100 steps: Average Loss 1.229 | Accuracy: 66%
[Step 700] Past 100 steps: Average Loss 1.098 | Accuracy: 68%
[Step 800] Past 100 steps: Average Loss 0.856 | Accuracy: 70%
[Step 900] Past 100 steps: Average Loss 0.838 | Accuracy: 75%
[Step 1000] Past 100 steps: Average Loss 0.692 | Accuracy: 79%
[Step 1100] Past 100 steps: Average Loss 0.706 | Accuracy: 78%
[Step 1200] Past 100 steps: Average Loss 0.713 | Accuracy: 77%
[Step 1300] Past 100 steps: Average Loss 0.562 | Accuracy: 84%
[Step 1400] Past 100 steps: Average Loss 0.562 | Accuracy: 82%
[Step 1500] Past 100 steps: Average Loss 0.475 | Accuracy: 86%
[Step 1600] Past 100 steps

[Step 13100] Past 100 steps: Average Loss 0.583 | Accuracy: 87%
[Step 13200] Past 100 steps: Average Loss 0.566 | Accuracy: 85%
[Step 13300] Past 100 steps: Average Loss 0.317 | Accuracy: 90%
[Step 13400] Past 100 steps: Average Loss 0.338 | Accuracy: 93%
[Step 13500] Past 100 steps: Average Loss 0.561 | Accuracy: 83%
[Step 13600] Past 100 steps: Average Loss 0.386 | Accuracy: 87%
[Step 13700] Past 100 steps: Average Loss 0.311 | Accuracy: 88%
[Step 13800] Past 100 steps: Average Loss 0.217 | Accuracy: 93%
[Step 13900] Past 100 steps: Average Loss 0.232 | Accuracy: 90%
[Step 14000] Past 100 steps: Average Loss 0.246 | Accuracy: 91%
[Step 14100] Past 100 steps: Average Loss 0.408 | Accuracy: 91%
[Step 14200] Past 100 steps: Average Loss 0.225 | Accuracy: 95%
[Step 14300] Past 100 steps: Average Loss 0.384 | Accuracy: 84%
[Step 14400] Past 100 steps: Average Loss 0.341 | Accuracy: 86%
[Step 14500] Past 100 steps: Average Loss 0.339 | Accuracy: 90%
[Step 14600] Past 100 steps: Average Los

[Step 26000] Past 100 steps: Average Loss 0.261 | Accuracy: 91%
[Step 26100] Past 100 steps: Average Loss 0.323 | Accuracy: 93%
[Step 26200] Past 100 steps: Average Loss 0.204 | Accuracy: 89%
[Step 26300] Past 100 steps: Average Loss 0.235 | Accuracy: 93%
[Step 26400] Past 100 steps: Average Loss 0.298 | Accuracy: 89%
[Step 26500] Past 100 steps: Average Loss 0.324 | Accuracy: 90%
[Step 26600] Past 100 steps: Average Loss 0.360 | Accuracy: 88%
[Step 26700] Past 100 steps: Average Loss 0.426 | Accuracy: 90%
[Step 26800] Past 100 steps: Average Loss 0.184 | Accuracy: 94%
[Step 26900] Past 100 steps: Average Loss 0.185 | Accuracy: 95%
[Step 27000] Past 100 steps: Average Loss 0.252 | Accuracy: 89%
[Step 27100] Past 100 steps: Average Loss 0.234 | Accuracy: 92%
[Step 27200] Past 100 steps: Average Loss 0.341 | Accuracy: 90%
[Step 27300] Past 100 steps: Average Loss 0.202 | Accuracy: 97%
[Step 27400] Past 100 steps: Average Loss 0.250 | Accuracy: 91%
[Step 27500] Past 100 steps: Average Los

[Step 38900] Past 100 steps: Average Loss 0.398 | Accuracy: 87%
[Step 39000] Past 100 steps: Average Loss 0.323 | Accuracy: 90%
[Step 39100] Past 100 steps: Average Loss 0.139 | Accuracy: 98%
[Step 39200] Past 100 steps: Average Loss 0.434 | Accuracy: 88%
[Step 39300] Past 100 steps: Average Loss 0.181 | Accuracy: 93%
[Step 39400] Past 100 steps: Average Loss 0.196 | Accuracy: 96%
[Step 39500] Past 100 steps: Average Loss 0.103 | Accuracy: 96%
[Step 39600] Past 100 steps: Average Loss 0.253 | Accuracy: 94%
[Step 39700] Past 100 steps: Average Loss 0.175 | Accuracy: 95%
[Step 39800] Past 100 steps: Average Loss 0.263 | Accuracy: 93%
[Step 39900] Past 100 steps: Average Loss 0.356 | Accuracy: 89%
[Step 40000] Past 100 steps: Average Loss 0.149 | Accuracy: 97%
[Step 40100] Past 100 steps: Average Loss 0.438 | Accuracy: 86%
[Step 40200] Past 100 steps: Average Loss 0.180 | Accuracy: 92%
[Step 40300] Past 100 steps: Average Loss 0.182 | Accuracy: 94%
[Step 40400] Past 100 steps: Average Los

[Step 9900] Past 100 steps: Average Loss 0.129 | Accuracy: 94%
[Step 10000] Past 100 steps: Average Loss 0.372 | Accuracy: 89%
[Step 10100] Past 100 steps: Average Loss 0.076 | Accuracy: 97%
[Step 10200] Past 100 steps: Average Loss 0.123 | Accuracy: 98%
[Step 10300] Past 100 steps: Average Loss 0.247 | Accuracy: 93%
[Step 10400] Past 100 steps: Average Loss 0.233 | Accuracy: 94%
[Step 10500] Past 100 steps: Average Loss 0.136 | Accuracy: 97%
[Step 10600] Past 100 steps: Average Loss 0.286 | Accuracy: 91%
[Step 10700] Past 100 steps: Average Loss 0.071 | Accuracy: 98%
[Step 10800] Past 100 steps: Average Loss 0.332 | Accuracy: 91%
[Step 10900] Past 100 steps: Average Loss 0.397 | Accuracy: 89%
[Step 11000] Past 100 steps: Average Loss 0.186 | Accuracy: 95%
[Step 11100] Past 100 steps: Average Loss 0.342 | Accuracy: 90%
[Step 11200] Past 100 steps: Average Loss 0.174 | Accuracy: 96%
[Step 11300] Past 100 steps: Average Loss 0.094 | Accuracy: 99%
[Step 11400] Past 100 steps: Average Loss

[Step 22800] Past 100 steps: Average Loss 0.173 | Accuracy: 93%
[Step 22900] Past 100 steps: Average Loss 0.135 | Accuracy: 98%
[Step 23000] Past 100 steps: Average Loss 0.294 | Accuracy: 94%
[Step 23100] Past 100 steps: Average Loss 0.376 | Accuracy: 91%
[Step 23200] Past 100 steps: Average Loss 0.173 | Accuracy: 94%
[Step 23300] Past 100 steps: Average Loss 0.354 | Accuracy: 89%
[Step 23400] Past 100 steps: Average Loss 0.220 | Accuracy: 92%
[Step 23500] Past 100 steps: Average Loss 0.160 | Accuracy: 95%
[Step 23600] Past 100 steps: Average Loss 0.505 | Accuracy: 92%
[Step 23700] Past 100 steps: Average Loss 0.358 | Accuracy: 91%
[Step 23800] Past 100 steps: Average Loss 0.381 | Accuracy: 89%
[Step 23900] Past 100 steps: Average Loss 0.080 | Accuracy: 97%
[Step 24000] Past 100 steps: Average Loss 0.214 | Accuracy: 90%
[Step 24100] Past 100 steps: Average Loss 0.201 | Accuracy: 91%
[Step 24200] Past 100 steps: Average Loss 0.279 | Accuracy: 93%
[Step 24300] Past 100 steps: Average Los

[Step 35700] Past 100 steps: Average Loss 0.259 | Accuracy: 93%
[Step 35800] Past 100 steps: Average Loss 0.305 | Accuracy: 90%
[Step 35900] Past 100 steps: Average Loss 0.257 | Accuracy: 92%
[Step 36000] Past 100 steps: Average Loss 0.299 | Accuracy: 90%
[Step 36100] Past 100 steps: Average Loss 0.314 | Accuracy: 91%
[Step 36200] Past 100 steps: Average Loss 0.054 | Accuracy: 98%
[Step 36300] Past 100 steps: Average Loss 0.272 | Accuracy: 94%
[Step 36400] Past 100 steps: Average Loss 0.099 | Accuracy: 97%
[Step 36500] Past 100 steps: Average Loss 0.141 | Accuracy: 96%
[Step 36600] Past 100 steps: Average Loss 0.223 | Accuracy: 94%
[Step 36700] Past 100 steps: Average Loss 0.204 | Accuracy: 94%
[Step 36800] Past 100 steps: Average Loss 0.267 | Accuracy: 96%
[Step 36900] Past 100 steps: Average Loss 0.318 | Accuracy: 91%
[Step 37000] Past 100 steps: Average Loss 0.410 | Accuracy: 88%
[Step 37100] Past 100 steps: Average Loss 0.391 | Accuracy: 90%
[Step 37200] Past 100 steps: Average Los

[Step 6600] Past 100 steps: Average Loss 0.435 | Accuracy: 94%
[Step 6700] Past 100 steps: Average Loss 0.413 | Accuracy: 89%
[Step 6800] Past 100 steps: Average Loss 0.249 | Accuracy: 97%
[Step 6900] Past 100 steps: Average Loss 0.429 | Accuracy: 91%
[Step 7000] Past 100 steps: Average Loss 0.252 | Accuracy: 91%
[Step 7100] Past 100 steps: Average Loss 0.175 | Accuracy: 95%
[Step 7200] Past 100 steps: Average Loss 0.330 | Accuracy: 89%
[Step 7300] Past 100 steps: Average Loss 0.331 | Accuracy: 90%
[Step 7400] Past 100 steps: Average Loss 0.238 | Accuracy: 92%
[Step 7500] Past 100 steps: Average Loss 0.334 | Accuracy: 94%
[Step 7600] Past 100 steps: Average Loss 0.600 | Accuracy: 88%
[Step 7700] Past 100 steps: Average Loss 0.480 | Accuracy: 89%
[Step 7800] Past 100 steps: Average Loss 0.344 | Accuracy: 92%
[Step 7900] Past 100 steps: Average Loss 0.197 | Accuracy: 92%
[Step 8000] Past 100 steps: Average Loss 0.312 | Accuracy: 93%
[Step 8100] Past 100 steps: Average Loss 0.227 | Accura

In [ ]:
result = model.predict(testing_images)
test_labels = []
for i in result:
    for j in range(len(i)):
        if i[j] == max(i):
            test_labels.append(j)

In [ ]:
fh = open('E:/Curious Dev B/Kaggle/results - Digit Recogniser(24).csv', 'w')
ct = 0
for i in test_labels:
    ct += 1
    fstr = str(ct)+','+str(i)+','+'\n'
    fh.write(fstr)

In [ ]:
from IPython.display import display
from PIL import Image
testing_images.shape = (28000, 28, 28)
for i in testing_images[27900:]:
    img = Image.fromarray(np.uint8((i)*255))
    display(img)